In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression

## Analysis of multivariate random variables

## Step 1. You need to make a non-parametric estimation of PDF in form of histogram and using kernel density function for MRV (or probability law in case of discrete MRV).

In [ ]:
fig = px.scatter_matrix(
    df,
    dimensions=df_ml,#["year_of_release", "na_sales", "eu_sales", "jp_sales", "other_sales"],
    color='genre'
)

fig.update_layout(
    title='Parametric estimation of PDF',
    width=1000,
    height=1000,
)

fig.show()

In [ ]:
df_ml = df.query('critic_score != "unknown"').copy()
df_ml = df_ml.query('user_score != "unknown"')
df_ml['critic_score'] = df_ml['critic_score'].astype('int')
df_ml['user_score'] = df_ml['user_score'].astype('float')

sns.set_theme(style='whitegrid', palette='pastel')

ax = sns.pairplot(
    df_ml,
    diag_kind='kde'
)
ax.map_lower(sns.kdeplot, levels=6, color='.1')

plt.show()

In [ ]:
sns.set_theme(style='whitegrid', palette='pastel')

ax = sns.pairplot(
    df_ml,
    kind="hist",
    diag_kind='hist'
)

plt.show()

In [ ]:
fig = px.density_heatmap(
    df,
    x = 'year_of_release',
    y = 'genre',
    # z = 'passengers',
    title = 'Non-parametric estimation of PDF', # Non-parametric estimation of PDF
    #     color_continuous_scale = px.colors.diverging.BrBG, # not to be used with marginal_x and marginal_y
    marginal_x = 'histogram',
    marginal_y = 'histogram'
)


fig.show()

## Step 2. You need to make an estimation of multivariate mathematical expectation and variance.

In [ ]:
display(df.describe().T)

In [ ]:
df.var()

## Step 3. You need to make a non-parametric estimation of conditional distributions, mathematical expectations and variances.

In [ ]:
figure, ax = plt.subplots(3, 2, figsize=(20, 15))
sns.set_theme(style='whitegrid', palette='husl')

year_of_release = sns.histplot(df.year_of_release, ax=ax[0, 0], kde=True, stat='density')
year_of_release.set(xlabel='Year Of Release')

na_sales = sns.histplot(df.na_sales, ax=ax[0, 1], kde=True, stat='density')
na_sales.set(xlabel='na_sales')

eu_sales = sns.histplot(df.eu_sales, ax=ax[1, 0], kde=True, stat='density')
eu_sales.set(xlabel='eu_sales')

jp_sales = sns.histplot(df.jp_sales, ax=ax[1, 1], kde=True, stat='density')
jp_sales.set(xlabel='jp_sales')

other_sales = sns.histplot(df.other_sales, ax=ax[2, 0], kde=True, stat='density')
other_sales.set(xlabel='other_sales')

world_sales = sns.histplot(df.world_sales, ax=ax[2, 1], kde=True, stat='density')
world_sales.set(xlabel='world_sales')

plt.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np
# np.random.seed(1)

x = df.year_of_release
hist_data = [df.year_of_release]
group_labels = ['Year Of Release'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels)
fig.show()

In [ ]:
x1 = df.na_sales
x2 = df.eu_sales
x3 = df.jp_sales
x4 = df.other_sales
x5 = df.world_sales

hist_data = [x1, x2, x3, x4, x5]

group_labels = ['na_sales', 'eu_sales', "jp_sales", 'other_sales', 'world_sales']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels)
fig.show()

In [ ]:
all_variables = [
    'name', 'platform', 'year_of_release', 'genre',
    'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'world_sales',
    'critic_score', 'user_score', 'rating',
]

conditions = [2000, 2005, 2010, 2015]


conditional_mean = pd.DataFrame([], index=all_variables)
conditional_var = pd.DataFrame([], index=all_variables)

for condition in conditions:
    tmp_df = df_ml[df_ml["year_of_release"] == condition][all_variables]
    conditional_mean[f"year_of_release = {condition}"] = tmp_df.mean(axis=0)
    conditional_var[f"year_of_release = {condition}"] = tmp_df.var(axis=0)

In [ ]:
conditional_mean

In [ ]:
conditional_var

## Step 4. You need to make an estimation of pair correlation coefficients, confidence intervals for them and significance levels.

In [ ]:
figure, ax = plt.subplots(1, 1, figsize=(10, 10))

sns.heatmap(df[['na_sales', 'jp_sales', 'eu_sales']].corr(), cmap='Blues', annot=True, linewidths=0.25)

plt.show()

In [ ]:
fig = px.imshow(
    df[['na_sales', 'jp_sales', 'eu_sales']].corr(),
    text_auto=True,
)

fig.show()

> Thus, sales in North America have a high correlation with sales in Japan

In [ ]:
import numpy as np, scipy.stats as st

def estimate(values, target):
    r, pvalue = scipy.stats.pearsonr(values, target)

    conf_int = st.t.interval(0.95, len(values)-1, loc=np.mean(values), scale=st.sem(values))

    print(f'Correlation Coefficient: {r} \nSignificance Level: {pvalue}\nConfidence Interval: {conf_int}')

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore')
ohe_data = ohe.fit_transform(df[['genre']])
ohe_data.toarray()

In [ ]:
estimate(df.jp_sales, df.na_sales)

## Step 5. Choose a task formulation for regression. Estimate multivariate correlation (target - predictors).

In [ ]:
pd.get_dummies(df['genre'])

In [ ]:
# df.info()
data = df.query('critic_score != "unknown" & rating == ["E", "T", "M"]')
data['critic_score'] = data['critic_score'].map(float) # astype(float)
# data['critic_score'].value_counts()
data['rating'].value_counts()
# data.info()
data = data[['critic_score', 'rating', 'genre', 'na_sales']]
# df['genre'] = pd.get_dummies(df['genre'])
data = pd.get_dummies(data, columns=['genre', 'rating'])
data

## Step 6. Build regression model and make an analysis of multicollinearity and regularization (if needed).

In [ ]:
from sklearn.model_selection import train_test_split
# Split data
df_X = data.drop(columns='critic_score')
df_y = data['na_sales']
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
# Teach regressor
cls_lr = LinearRegression()
cls_lr.fit(X_train, y_train)

In [ ]:
# Predict
y_pred = cls_lr.predict(X_test)
y_pred

In [ ]:
from sklearn import linear_model

# Lasso regularization
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X_train, y_train)
print(clf.coef_)

In [ ]:
from sklearn.linear_model import LassoLarsIC

model_aic = LassoLarsIC(criterion='aic')
model_aic.fit(X_train, y_train)
alpha_aic_ = model_aic.coef_
alpha_aic_

In [ ]:
y_pred_lasso = clf.predict(X_test)
y_pred_lasso_aic = model_aic.predict(X_test)
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
mae_lasso_aic = mean_absolute_error(y_test, y_pred_lasso_aic)
print('Mean absolute error with lasso = ', mae_lasso)
print('Mean squared error with lasso = ', mse_lasso)
print('Mean absolute error with aic lasso = ', mae_lasso_aic)

## Step 7. Analyze the quality of regression model (distribution of residuals, determination coefficient).

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

def mae(y_test, y_pred):
    print(f"MAE {mean_absolute_error(y_test, y_pred)}\n")


def mape(y_test, y_pred):
    print(f"MAPE {mean_absolute_percentage_error(y_test, y_pred)}\n")

def mse(y_test, y_pred):
    print(f"MSE {mean_squared_error(y_test, y_pred)}\n")

def rmse(y_test, y_pred):
    # squaredbool, default=True
    # If True returns MSE value, if False returns RMSE value.
    print(f"RMSE {mean_squared_error(y_test, y_pred, squared=False)}\n")

def residuals_dist(y_test, y_pred):

    data = y_test - y_pred

    figure, ax = plt.subplots(1, 1, figsize=(10, 10))

    residuals = sns.histplot(data, ax=ax, kde=True, stat='density')
    residuals.set(xlabel='Distribution of Residuals')

    plt.show()

def get_regression_metrics(y_test, y_pred):
    mae(y_test, y_pred)
    mape(y_test, y_pred)
    mse(y_test, y_pred)
    rmse(y_test, y_pred)
    residuals_dist(y_test, y_pred)

    r2 = r2_score(y_test, y_pred)
    print('r2 score ', r2)

In [ ]:
get_regression_metrics(y_test, y_pred)

In [ ]:
#Confidence interval of regression coef
import numpy as np, statsmodels.api as sm
mod = sm.OLS(y_train, X_train)
res = mod.fit()
print(res.conf_int(0.01))